In [2]:
import psycopg2
import random

In [4]:
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="P0stGr3sM6ntr6",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

##########################################
####         CLEAR TABLES             ####
##########################################

In [41]:


cursor.execute("UPDATE item_loc_soh SET stock_on_hand =0;")

cursor.execute("DELETE FROM event;")

cursor.execute("DELETE FROM receipts;")

conn.commit()
cursor.close()
conn.close()


##########################################
#### DC RECEIPT TRANSACTION GENERATOR ####
##########################################

In [58]:

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="P0stGr3sM6ntr6",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# Parameters for receipts creation
num_receipts = 1000  # Number of receipts to create
max_units_received = 1000  # Maximum number of units in each receipt

# Get product, location IDs, and stock_on_hand from Item_Loc_SOH for locations with type 'W'
cursor.execute("""
    SELECT product_id, location_id, stock_on_hand
    FROM item_loc_soh
    WHERE location_id IN (SELECT location_id FROM location WHERE location_type = 'W');
""")
records = cursor.fetchall()

# Current date and timestamp for the Event table
cursor.execute("SELECT CURRENT_DATE;")
current_date = cursor.fetchone()[0]

cursor.execute("SELECT CURRENT_TIMESTAMP;")
current_timestamp = cursor.fetchone()[0]

# Create random receipt entries
for _ in range(num_receipts):
    # Select a random product-location record
    product_id, location_id, stock_on_hand = random.choice(records)
    units_received = random.randint(1, max_units_received)

    # Insert a record into the Receipts table
    cursor.execute(
        """
        INSERT INTO Receipts (product_id, location_id, number_of_units_received, receipt_cost)
        VALUES (%s, %s, %s, %s)
        RETURNING receipt_cost
        """,
        (product_id, location_id, units_received, random.uniform(10, 100))
    )
    receipt_cost = cursor.fetchone()[0]

    # Insert a record into the Event table, using stock_on_hand as actual_soh
    cursor.execute(
        """
        INSERT INTO Event (event_type, product_id, location_id, soh_impact, actual_soh, transaction_date, transaction_time)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """,
        ('receipt', product_id, location_id, units_received, stock_on_hand, current_date, current_timestamp)
    )

    # Update stock_on_hand in Item_Loc_SOH table
    cursor.execute(
        """
        UPDATE Item_Loc_SOH
        SET stock_on_hand = stock_on_hand + %s
        WHERE product_id = %s AND location_id = %s
        """,
        (units_received, product_id, location_id)
    )

# Commit and close
conn.commit()
cursor.close()
conn.close()

##########################################
####  ALLOCTION TRANSACTION GENERATOR ####
##########################################

In [64]:
import psycopg2
from datetime import datetime

# --- Configuration ---
DB_CONFIG = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "P0stGr3sM6ntr6",
    "host": "localhost",
    "port": 5432
}

allocation_qty_per_store = 5
max_allocations = 10  # <--  how many allocations

# --- Allocation Harness ---
def allocate_products_from_dc():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    try:
        # Step 1: Get eligible products from warehouse (DC) with sufficient stock
        cur.execute("""
            SELECT ils.product_id, ils.location_id AS dc_location_id, ils.stock_on_hand, ils.reserved_quantity
            FROM Item_Loc_SOH ils
            JOIN Location l ON ils.location_id = l.location_id
            WHERE l.location_type = 'W' AND ils.stock_on_hand > 500
            LIMIT %s;
        """, (max_allocations,))
        eligible_rows = cur.fetchall()

        if not eligible_rows:
            print("No eligible DC products found with sufficient stock.")
            return

        now = datetime.now()
        today = now.date()

        for product_id, dc_location_id, stock_on_hand, dc_reserved_before in eligible_rows:
            required_qty = allocation_qty_per_store * 10

            if stock_on_hand < required_qty + 100:
                print(f"Skipping product {product_id} at DC {dc_location_id} due to insufficient stock.")
                continue

            # Step 2: Get 10 store location_ids dynamically
            cur.execute("""
                SELECT location_id FROM Location
                WHERE location_type = 'S'
                ORDER BY RANDOM()
                LIMIT 10;
            """)
            store_location_ids = [row[0] for row in cur.fetchall()]

            if len(store_location_ids) < 10:
                print(f"Skipping product {product_id}: Less than 10 stores available.")
                continue

            print(f"\nAllocating product {product_id} from DC {dc_location_id} to 10 stores...")

            # Step 3: Insert allocation entries
            for store_id in store_location_ids:
                cur.execute("""
                    INSERT INTO ALLOCATION_ITEM_DETAIL (
                        from_loc, to_loc, product_id, quantity_allocated, quantity_shipped,
                        quantity_received, create_datetime, last_update_datetime
                    ) VALUES (%s, %s, %s, %s, 0, 0, %s, %s);
                """, (
                    dc_location_id, store_id, product_id,
                    allocation_qty_per_store, now, now
                ))

            # Step 4: Update reserved_quantity at DC
            cur.execute("""
                UPDATE Item_Loc_SOH
                SET reserved_quantity = reserved_quantity + %s
                WHERE product_id = %s AND location_id = %s;
            """, (required_qty, product_id, dc_location_id))

            # Step 5: Insert event for DC
            cur.execute("""
                INSERT INTO event (
                    event_type, product_id, location_id,
                    reserved_quantity, actual_reserved_quantity,
                    transaction_date, transaction_time
                ) VALUES (
                    'ALLOCATION_CREATION', %s, %s, %s, %s, %s, %s
                );
            """, (
                product_id, dc_location_id,
                required_qty, dc_reserved_before,
                today, now
            ))

            # Step 6: Upsert expected_qty at each store + Insert event
            for store_id in store_location_ids:
                # Get expected_qty before upsert
                cur.execute("""
                    SELECT expected_qty FROM Item_Loc_SOH
                    WHERE product_id = %s AND location_id = %s;
                """, (product_id, store_id))
                row = cur.fetchone()
                expected_before = row[0] if row else 0

                # Upsert
                cur.execute("""
                    INSERT INTO Item_Loc_SOH (
                        product_id, location_id, stock_on_hand, average_cost,
                        reserved_quantity, non_sellable_quantity, in_transit_qty, expected_qty
                    )
                    VALUES (%s, %s, 0, 0.0, 0, 0, 0, %s)
                    ON CONFLICT (product_id, location_id)
                    DO UPDATE SET expected_qty = Item_Loc_SOH.expected_qty + EXCLUDED.expected_qty;
                """, (product_id, store_id, allocation_qty_per_store))

                # Insert event for store
                cur.execute("""
                    INSERT INTO event (
                        event_type, product_id, location_id,
                        expected_quantity, actual_expected_quantity,
                        transaction_date, transaction_time
                    ) VALUES (
                        'ALLOCATION_CREATION', %s, %s, %s, %s, %s, %s
                    );
                """, (
                    product_id, store_id,
                    allocation_qty_per_store, expected_before,
                    today, now
                ))

            print(f"✅ Allocation done for product {product_id}.")

        conn.commit()
        print("\n✅ All allocations (up to limit) processed successfully.")

    except Exception as e:
        conn.rollback()
        print("❌ Error during allocation:", e)

    finally:
        cur.close()
        conn.close()


# --- Run the Allocation Process ---
allocate_products_from_dc()


Allocating product 30000073 from DC 1 to 10 stores...
✅ Allocation done for product 30000073.

Allocating product 30000359 from DC 1 to 10 stores...
✅ Allocation done for product 30000359.

Allocating product 30000083 from DC 1 to 10 stores...
✅ Allocation done for product 30000083.

Allocating product 30001432 from DC 1 to 10 stores...
✅ Allocation done for product 30001432.

Allocating product 30000893 from DC 1 to 10 stores...
✅ Allocation done for product 30000893.

Allocating product 30000694 from DC 1 to 10 stores...
✅ Allocation done for product 30000694.

Allocating product 30000369 from DC 1 to 10 stores...
✅ Allocation done for product 30000369.

Allocating product 30000717 from DC 1 to 10 stores...
✅ Allocation done for product 30000717.

Allocating product 30000921 from DC 1 to 10 stores...
✅ Allocation done for product 30000921.

Allocating product 30000827 from DC 1 to 10 stores...
✅ Allocation done for product 30000827.

✅ All allocations (up to limit) processed success

##########################################

ALLOCATION SHIPMENT

##########################################

In [62]:
import psycopg2
from datetime import datetime
import random

# --- Configuration ---
DB_CONFIG = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "P0stGr3sM6ntr6",
    "host": "localhost",
    "port": 5432
}

def generate_random_number(length=10):
    return ''.join(random.choices('0123456789', k=length))

def process_shipments_for_allocations():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    try:
        # Step 1: Get open allocations
        cur.execute("""
            SELECT allocation_id, from_loc, to_loc, product_id, quantity_allocated, quantity_shipped
            FROM ALLOCATION_ITEM_DETAIL
            WHERE quantity_allocated > quantity_shipped;
        """)
        open_allocations = cur.fetchall()

        if not open_allocations:
            print("No open allocations to process.")
            return

        bol_number = generate_random_number()
        now = datetime.now()
        today = now.date()

        print(f"\n🚚 Creating shipment with BOL Number: {bol_number}")

        for allocation in open_allocations:
            allocation_id, from_loc, to_loc, product_id, quantity_allocated, quantity_shipped = allocation
            qty_to_ship = quantity_allocated - quantity_shipped
            carton_number = generate_random_number()
            distro_number = allocation_id  # ✅ Now using allocation_id

            print(f"Processing Allocation ID {allocation_id} for Product {product_id} | Qty: {qty_to_ship}")

            # Step 1: Get current DC values
            cur.execute("""
                SELECT stock_on_hand, reserved_quantity
                FROM Item_Loc_SOH
                WHERE product_id = %s AND location_id = %s;
            """, (product_id, from_loc))
            dc_soh, dc_reserved = cur.fetchone()

            # Step 2: Get current Store values
            cur.execute("""
                SELECT expected_qty, in_transit_qty
                FROM Item_Loc_SOH
                WHERE product_id = %s AND location_id = %s;
            """, (product_id, to_loc))
            store_expected, store_in_transit = cur.fetchone()

            # Step 3: Update DC - reduce reserved and stock_on_hand
            cur.execute("""
                UPDATE Item_Loc_SOH
                SET 
                    reserved_quantity = GREATEST(reserved_quantity - %s, 0),
                    stock_on_hand = GREATEST(stock_on_hand - %s, 0)
                WHERE product_id = %s AND location_id = %s;
            """, (qty_to_ship, qty_to_ship, product_id, from_loc))

            # Step 4: Update Store - move expected → in_transit
            cur.execute("""
                UPDATE Item_Loc_SOH
                SET 
                    expected_qty = GREATEST(expected_qty - %s, 0),
                    in_transit_qty = in_transit_qty + %s
                WHERE product_id = %s AND location_id = %s;
            """, (qty_to_ship, qty_to_ship, product_id, to_loc))

            # Step 5: Update allocation_item_detail
            cur.execute("""
                UPDATE ALLOCATION_ITEM_DETAIL
                SET quantity_shipped = quantity_shipped + %s,
                    last_update_datetime = %s
                WHERE allocation_id = %s;
            """, (qty_to_ship, now, allocation_id))

            # Step 6: Insert into shipment_item_detail with allocation_id as distro_number
            cur.execute("""
                INSERT INTO shipment_item_detail (
                    distro_type, bol_number, carton_number, status, 
                    product_id, quantity, create_datetime, last_update_datetime,
                    distro_number, from_loc, to_loc
                )
                VALUES ('A', %s, %s, 'S', %s, %s, %s, %s, %s, %s, %s);
            """, (
                bol_number, carton_number, product_id, qty_to_ship,
                now, now, distro_number, from_loc, to_loc
            ))

            # Step 7: Event for DC
            cur.execute("""
                INSERT INTO event (
                    event_type, product_id, location_id,
                    soh_impact, actual_soh, reserved_quantity, actual_reserved_quantity,
                    transaction_date, transaction_time
                ) VALUES (
                    'ALLOCATION_SHIPMENT', %s, %s,
                    %s, %s, %s, %s,
                    %s, %s
                );
            """, (
                product_id, from_loc,
                -qty_to_ship, dc_soh,
                -qty_to_ship, dc_reserved,
                today, now
            ))

            # Step 8: Event for Store
            cur.execute("""
                INSERT INTO event (
                    event_type, product_id, location_id,
                    expected_quantity, actual_expected_quantity,
                    in_transit_quantity, actual_in_transit_quantity,
                    transaction_date, transaction_time
                ) VALUES (
                    'ALLOCATION_SHIPMENT', %s, %s,
                    %s, %s,
                    %s, %s,
                    %s, %s
                );
            """, (
                product_id, to_loc,
                -qty_to_ship, store_expected,
                qty_to_ship, store_in_transit,
                today, now
            ))

        conn.commit()
        print("\n✅ Shipment and event records created successfully.")

    except Exception as e:
        conn.rollback()
        print("❌ Error during shipment processing:", e)

    finally:
        cur.close()
        conn.close()


# --- Run the Shipment Harness ---
process_shipments_for_allocations()


🚚 Creating shipment with BOL Number: 6364899418
Processing Allocation ID 41 for Product 30000122 | Qty: 5
Processing Allocation ID 42 for Product 30000122 | Qty: 5
Processing Allocation ID 43 for Product 30000122 | Qty: 5
Processing Allocation ID 44 for Product 30000122 | Qty: 5
Processing Allocation ID 45 for Product 30000122 | Qty: 5
Processing Allocation ID 46 for Product 30000122 | Qty: 5
Processing Allocation ID 47 for Product 30000122 | Qty: 5
Processing Allocation ID 48 for Product 30000122 | Qty: 5
Processing Allocation ID 49 for Product 30000122 | Qty: 5
Processing Allocation ID 50 for Product 30000122 | Qty: 5
Processing Allocation ID 51 for Product 30000073 | Qty: 5
Processing Allocation ID 52 for Product 30000073 | Qty: 5
Processing Allocation ID 53 for Product 30000073 | Qty: 5
Processing Allocation ID 54 for Product 30000073 | Qty: 5
Processing Allocation ID 55 for Product 30000073 | Qty: 5
Processing Allocation ID 56 for Product 30000073 | Qty: 5
Processing Allocation I

##########################################

ALLOCATION RECEIPTS

##########################################

In [54]:
import psycopg2
from datetime import datetime

# --- Configuration ---
DB_CONFIG = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "P0stGr3sM6ntr6",
    "host": "localhost",
    "port": 5432
}

def process_allocation_receipts():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    try:
        # Step 1: Get all 'S' status shipment lines
        cur.execute("""
            SELECT 
                shipment_id, distro_number, product_id, quantity, to_loc
            FROM shipment_item_detail
            WHERE status = 'S';
        """)
        shipments = cur.fetchall()

        if not shipments:
            print("No pending shipment receipts.")
            return

        now = datetime.now()
        today = now.date()

        for shipment_id, allocation_id, product_id, quantity_shipped, store_loc in shipments:
            # Step 2: Get allocation to check if receipt is needed
            cur.execute("""
                SELECT quantity_received, quantity_shipped
                FROM ALLOCATION_ITEM_DETAIL
                WHERE allocation_id = %s AND product_id = %s;
            """, (allocation_id, product_id))
            result = cur.fetchone()

            if not result:
                print(f"⚠️ Allocation ID {allocation_id} not found. Skipping.")
                continue

            quantity_received, allocation_shipped = result

            if quantity_received >= allocation_shipped:
                print(f"✅ Already fully received for Allocation ID {allocation_id}. Skipping.")
                continue

            print(f"\n📦 Processing receipt for Allocation ID {allocation_id} | Product {product_id} | Qty: {quantity_shipped}")

            # Step 3.1: Update allocation received quantity
            cur.execute("""
                UPDATE ALLOCATION_ITEM_DETAIL
                SET quantity_received = quantity_received + %s,
                    last_update_datetime = %s
                WHERE allocation_id = %s AND product_id = %s;
            """, (quantity_shipped, now, allocation_id, product_id))

            # Step 3.2: Update shipment status to 'R'
            cur.execute("""
                UPDATE shipment_item_detail
                SET status = 'R',
                    last_update_datetime = %s
                WHERE shipment_id = %s;
            """, (now, shipment_id))

            # Step 3.3: Get current SOH at store
            cur.execute("""
                SELECT stock_on_hand, in_transit_qty
                FROM Item_Loc_SOH
                WHERE product_id = %s AND location_id = %s;
            """, (product_id, store_loc))
            row = cur.fetchone()
            if not row:
                print(f"❌ No Item_Loc_SOH record found for Product {product_id} at Store {store_loc}. Skipping.")
                continue

            store_stock, store_in_transit = row

            # Step 3.4: Move in_transit → stock_on_hand
            cur.execute("""
                UPDATE Item_Loc_SOH
                SET 
                    in_transit_qty = GREATEST(in_transit_qty - %s, 0),
                    stock_on_hand = stock_on_hand + %s
                WHERE product_id = %s AND location_id = %s;
            """, (quantity_shipped, quantity_shipped, product_id, store_loc))

            # Step 3.5: Insert event for store receipt
            cur.execute("""
                INSERT INTO event (
                    event_type, product_id, location_id,
                    in_transit_quantity, actual_in_transit_quantity,
                    soh_impact, actual_soh,
                    transaction_date, transaction_time
                )
                VALUES (
                    'ALLOCATION_RECEIPT', %s, %s,
                    %s, %s,
                    %s, %s,
                    %s, %s
                );
            """, (
                product_id, store_loc,
                -quantity_shipped, store_in_transit,
                quantity_shipped, store_stock,
                today, now
            ))

        conn.commit()
        print("\n✅ Receipt processing (with SOH update) completed successfully.")

    except Exception as e:
        conn.rollback()
        print("❌ Error during receipt processing:", e)

    finally:
        cur.close()
        conn.close()


# --- Run the Receipt Harness ---
process_allocation_receipts()


📦 Processing receipt for Allocation ID 1 | Product 30000073 | Qty: 5

📦 Processing receipt for Allocation ID 2 | Product 30000073 | Qty: 5

📦 Processing receipt for Allocation ID 3 | Product 30000073 | Qty: 5

📦 Processing receipt for Allocation ID 4 | Product 30000073 | Qty: 5

📦 Processing receipt for Allocation ID 5 | Product 30000073 | Qty: 5

📦 Processing receipt for Allocation ID 6 | Product 30000073 | Qty: 5

📦 Processing receipt for Allocation ID 7 | Product 30000073 | Qty: 5

📦 Processing receipt for Allocation ID 8 | Product 30000073 | Qty: 5

📦 Processing receipt for Allocation ID 9 | Product 30000073 | Qty: 5

📦 Processing receipt for Allocation ID 10 | Product 30000073 | Qty: 5

📦 Processing receipt for Allocation ID 11 | Product 30001432 | Qty: 5

📦 Processing receipt for Allocation ID 12 | Product 30001432 | Qty: 5

📦 Processing receipt for Allocation ID 13 | Product 30001432 | Qty: 5

📦 Processing receipt for Allocation ID 14 | Product 30001432 | Qty: 5

📦 Processing r

##########################################

SALES GENERATOR

##########################################

In [30]:

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="P0stGr3sM6ntr6",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# Parameters for receipts creation
num_sales = 50  # Number of receipts to create
max_units_sold = 3  # Maximum number of units in each sales

# Get product, location IDs, and stock_on_hand from Item_Loc_SOH for locations with type 'S'
cursor.execute("""
    SELECT product_id, location_id, stock_on_hand
    FROM item_loc_soh
    WHERE location_id IN (SELECT location_id FROM location WHERE location_type = 'S')
    AND stock_on_hand > 3;
""")
records = cursor.fetchall()

# Current date and timestamp for the Event table
cursor.execute("SELECT CURRENT_DATE;")
current_date = cursor.fetchone()[0]

cursor.execute("SELECT CURRENT_TIMESTAMP;")
current_timestamp = cursor.fetchone()[0]

# Create random sales entries
for _ in range(num_sales):
    # Select a random product-location record
    product_id, location_id, stock_on_hand,  = random.choice(records)
    units_sold = random.randint(1, max_units_sold)

    cursor.execute("SELECT unit_retail FROM product WHERE product_id = %s;", (product_id,))
    sales_retail = cursor.fetchone()[0]

    # Insert a record into the Sales table
    cursor.execute(
        """
        INSERT INTO Sales (product_id, location_id, number_of_units_sold, sales_retail, sales_type)
        VALUES (%s, %s, %s, %s, %s)
        """,
        (product_id, location_id, units_sold, sales_retail, 'FP')
    )

    # Insert a record into the Event table, using stock_on_hand as actual_soh
    cursor.execute(
        """
        INSERT INTO Event (event_type, product_id, location_id, soh_impact, actual_soh, transaction_date, transaction_time)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """,
        ('sale', product_id, location_id, units_sold, stock_on_hand, current_date, current_timestamp)
    )

    # Update stock_on_hand in Item_Loc_SOH table
    cursor.execute(
        """
        UPDATE Item_Loc_SOH
        SET stock_on_hand = stock_on_hand - %s
        WHERE product_id = %s AND location_id = %s
        """,
        (units_sold, product_id, location_id)
    )

# Commit and close
conn.commit()
cursor.close()
conn.close()